Building a basic agent 

creating a simple agent that can answer questions and call tools. The agent will use Claude Sonnet 4.5 as its language model, a basic weather function as a tool, and a simple prompt to guide its behavior.

Import Libraries 

In [12]:
from dotenv import load_dotenv
import os
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_anthropic import ChatAnthropic
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chat_models import init_chat_model
load_dotenv()

True

In [8]:
# Test the model initialization
model = init_chat_model("gpt-4o-mini")
response = model.invoke("What is the capital of France?")
response.content  

'The capital of France is Paris.'

Creating a simple reflection agent : An agent that generates an answer and then improves on that answer through a self critique 

In [15]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# LLMs
model = ChatOpenAI(model="gpt-4.1", temperature=0.7)
reflect_model = ChatOpenAI(model="gpt-4.1", temperature=0.2)

# Nodes
def answer_node(state):
    question = state["question"]
    answer = model.invoke(f"Answer this question:\n{question}")
    return {"draft_answer": answer.content}

def reflection_node(state):
    draft = state["draft_answer"]
    critique = reflect_model.invoke(
        f"You are a critical reviewer. Critique the answer below. "
        f"Say 'APPROVE' if good, otherwise suggest improvements.\n\nAnswer:\n{draft}"
    ).content
    return {"critique": critique}

def revise_node(state):
    draft = state["draft_answer"]
    critique = state["critique"]
    improved = model.invoke(
        f"Improve the answer using the critique.\n\nDraft:\n{draft}\n\nCritique:\n{critique}"
    ).content
    return {"draft_answer": improved}


# State
class AgentState(dict):
    question: str
    draft_answer: str
    critique: str

# Graph
graph = StateGraph(AgentState)
graph.add_node("answer", answer_node)
graph.add_node("reflect", reflection_node)
graph.add_node("revise", revise_node)

graph.set_entry_point("answer")
graph.add_edge("answer", "reflect")

def should_revise(state):
    return "APPROVE" not in state["critique"].upper()

graph.add_conditional_edges(
    "reflect",
    should_revise,
    {
        True: "revise",
        False: END
    },
)

graph.add_edge("revise", "reflect")

# ✅ Use a checkpointer
memory = MemorySaver()
agent = graph.compile(checkpointer=memory)

# ✅ Now you can invoke
result = agent.invoke(
    {"question": "Explain transformers in simple terms."},
    config={"thread_id": "run-1"}
)

print(result["draft_answer"])

Sure! In simple terms:

A **transformer** is a type of computer program used in artificial intelligence, especially for understanding and generating language (like how ChatGPT works).

- **How it works:** Imagine you’re reading a sentence. A transformer looks at all the words in that sentence at the same time, figuring out how each word relates to the others. It uses something called **"attention"** to focus on the important words when making sense or coming up with a response.
- **Why it’s special:** Older programs would read sentences word by word, but transformers can look at everything at once, which helps them understand context much better.

So, transformers help computers understand language more like humans do, by paying attention to all the important parts of what’s being said.
